In [1]:
api_key = "RGAPI-47322cd4-0159-48ab-a5c9-34df1fcbe14b"

In [2]:
api_url = "https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/colin%20ross"

In [3]:
import requests

In [4]:
api_url = api_url + '?api_key=' + api_key

In [5]:
resp = requests.get(api_url)

In [6]:
player_info = resp.json()

In [7]:
player_account_id = player_info['accountId']

In [8]:
api_url = "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/FVLlTJIenpB__DwELloJIJz4S7lrR29uw67Wl_6G9C1iYRE5jt4FxbDC4cPe4A17GmD9DfyimzJlww/ids?start=0&count=20"

In [9]:
api_url = api_url + "&api_key=" + api_key

In [10]:
api_url

'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/FVLlTJIenpB__DwELloJIJz4S7lrR29uw67Wl_6G9C1iYRE5jt4FxbDC4cPe4A17GmD9DfyimzJlww/ids?start=0&count=20&api_key=RGAPI-47322cd4-0159-48ab-a5c9-34df1fcbe14b'

In [11]:
resp = requests.get(api_url)

In [12]:
matches = resp.json()

In [13]:
match = matches[0]

In [14]:
match

'NA1_4908910130'

In [15]:
api_url = "https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4908910130"

In [16]:
api_url = api_url + "?api_key=" + api_key

In [17]:
api_url

'https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4908910130?api_key=RGAPI-47322cd4-0159-48ab-a5c9-34df1fcbe14b'

In [18]:
resp = requests.get(api_url)

In [19]:
match_data = resp.json()

In [20]:
match_data.keys()

dict_keys(['metadata', 'info'])

In [21]:
puuid = player_info['puuid']

In [22]:
part_index = match_data['metadata']['participants'].index(puuid)

In [23]:
match_data['info']['participants'][part_index]['assists']

5

In [24]:
def get_match_data(region, match_id, api_key):
    api_url = (
        "https://" +
        region +
        ".api.riotgames.com/lol/match/v5/matches/" +
        match_id +
        "?api_key=" +
        api_key
    )
    
    resp = requests.get(api_url)
    data = resp.json()
    return data

In [25]:
region = 'americas'
match_id = 'NA1_4908910130'
api_key = "RGAPI-47322cd4-0159-48ab-a5c9-34df1fcbe14b"

get_match_data(region, match_id, api_key)

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_4908910130',
  'participants': ['q5nDlMuc1AIJ2MzyqE7tJDdSZabw0BZyvUZ3IP1SQm2By86argaYhh86dBp4LBip5CJtHZkqVY8uyQ',
   'zYd8qv5jKrBj9St50naOO3pBsRlm6xF2dg-iJ22eX4ObYTRVHM66FoGWiXtPEWYh8qm4SIn3_VqKXA',
   'UDoIlfbTSP5V97-lfsAbM8mzpk57Wdld7PTBvI7vcMBd2WJnsDnk3rB5LeE2yuQRMV_80mYyFnl3xQ',
   'K7sp8IkDXdH8SytQK1hd1j3YaoTUjy7ipU1xCV2wQJxHcgwbjlBAdGNZMQtOlGTLtgP_3_350niR3w',
   '-Q9pSWwA3MjPpTKeh-G9fLxWQxak91YA9QxHInvBTCP-nYrzDfeFem4GzKbw_HK0uoT78FKpb8nEDA',
   'ZtLIUn6deM4333Zg69uqfnGPYWODSoy2IUtLqeLuMnr-Zo0elgvR0HkUuTAkpoOEikKs3FlZLL3uUQ',
   'id7akiIdpZUTnCaB7URP8TLLxXRRiZ1O35PWVVh1zFwj133LMnCviVm-glFO7n394do8v5HdHGPSIQ',
   'FVLlTJIenpB__DwELloJIJz4S7lrR29uw67Wl_6G9C1iYRE5jt4FxbDC4cPe4A17GmD9DfyimzJlww',
   'GskRbzIDh9Kh1RMTeEBn0HmY8DxeArvKzwrT4fCYTIK_SXcd3dKvr2N7biAaqL1tz-eIsgJkFcgTZQ',
   'owExeyJZVVtW82WHqXexAsTwlLrnNAfIvzVWKjRfslU0uoA-B7tM9FclBy-aYcIqDMh83cXNldfyUQ']},
 'info': {'gameCreation': 1706756970970,
  'gameDuration': 1816,
  